### Para crear imagenes
1) Importar pickle con datos de 1000 datos por segundo

2) Ver los segundos del arduino para poder realizar el acople de tiempo con el video. En realidad solo importa saber en que momento de la señal se realiza el primer pulso de este microcontrolador. 

3) Usar la transformada continua de Wavelets, investigar si usar Morlet. La libreria es pywt.

4) A partir del primer segundo del arduino, empezar a generar imagenes traslapadas en 300 ms de los canales elegidos y cantidad de datos por UN segundo. Para ello, se ocupa simplemente el metodo de Sliding Window. 

In [1]:
%matplotlib inline
#%matplotlib notebook
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import scipy
from scipy import signal
from PIL import Image
from tqdm import tqdm
import pywt
from glob2 import glob
import OpenEphys as OE
import os

In [2]:
# colocar la ruta y el nombre del archivo pickle generado con el codigo 01_ObtenerLFP

# RAT MILO - CONTROL
#fname = 'Raw_LFP_RAT02_SP_7_2017_01_11_18_46_49'
#ruta ='/home/romulo/DATA/Carolina/R02/RAT02_SP_7_2017_01_11_18_46_49/'

# RAT C14 ALEX
# Para utilizar pickle de 256 samples
# fname = 'DownsampledLFP256Hz_CA_EC03_C14'
# para utilizar pickle con 1000 samples
fname ='1kSLFPBandPass-3-100-Filtered_CA_EC03_C14'
ruta = '/home/cmanalisis/1_SergioEhijo/Ratas/C14_Alex/Pickles/'
lfp = pickle.load(open( ruta+fname+'.pickle', "rb" ) )

In [3]:
lfp

,Channel,Area,OriginalSampleRate,SampleRate,Use,Timestamps at original sample rate,LFP
0,CH1,lCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.018332029390342477, -0.03278736868023401, -..."
1,CH10,lSTR,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.04078297544615877, 0.10535131184173088, 0.1..."
2,CH11,lSTR,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[-0.07928274331206221, -0.1251439550652389, -0..."
3,CH12,lSTR,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.08537699764353247, 0.17523043936996005, 0.2..."
4,CH13,lCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[-0.15585135682155143, -0.16095390849804947, -..."
5,CH14,lCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.039891641245123094, 0.0939481313330054, 0.1..."
6,CH15,lCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.03284943558604545, -0.019316886556314868, -..."
7,CH16,lCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.06712764944191532, 0.13652784635475387, 0.1..."
8,CH17,rCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.05683469066840138, 0.08724999959020364, 0.1..."
9,CH18,rCFA,30000,1000.0,1,"[1294592.0, 1295616.0, 1296640.0, 1297664.0, 1...","[0.006086427542466597, 0.03293791354523316, 0...."


### Vamos a comparar los arreglos de tiempo entre el arduino y la señal de LFPs

### Por inspección visual, para alex el timestamp corresponde a 637.709 => index = np.argwhere(t2==stamp)[0]   (Este es para 30KHz, no sirve)
Hay que hacer un downsampling hasta 1Khz de la señal de arduino, y con eso se tiene el mismo tiempo real del arduino y lfps. Con esto, el tiempo importante es 21.258 segundos. La busqueda del index es equivalente.


In [4]:
dt = 0.001
timeLFP = np.arange(dt,len(lfp.LFP[1])*dt + dt, dt)
#stamp = 21.258
#index = np.where(timeLFP==stamp)
#index = np.argwhere(t2==on_ts[4])[0]
index= 21257
#plt.plot(timeLFP[index:], lfp.LFP[0][index:])
print(len(lfp.LFP[0][index:]))
#print(index*100 /len(lfp.LFP[0]))
index

2492663


21257

### Metodo de Sliding Window

In [5]:
def slidingWindow(sequence,winSize,step=1):
    """Returns a generator that will iterate through
    the defined chunks of input sequence.  Input sequence
    must be iterable."""
 
    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # Pre-compute number of chunks to emit
    # Es importante que numOfChunks sea int. Por ende la operacion divmod seapara en parte entera y decimal, con 
    # el segundo argumento (1) que indica la cantidad de decimales de d-
    numOfChunks,d = divmod(((len(sequence)-winSize)/ step)+1,1)
    #As int
    numOf = int(numOfChunks)
    # Do the work
    myWindows = []
    for i in range(0,numOf*step,step):
        #yield sequence[i:i+winSize]
        myWindows.append(sequence[i:i+winSize])
    return myWindows
    # Para el caso particular de considerar el ultimo pedazo de tiempo que puede ser mas corto debido a que la 
    # ventana no lo cubre, se tiene el siguiente if. Si no es necesario, comentar. 
    #if (d!=0):
     #   yield sequence[numOf*step:]

### Wave2Image: recibe la ventana, calcula la transformada wavelet y retorna la figura

In [6]:
def wave2Image(signal):
    """
    Esta funcion recibe tres parametros: la carpeta donde dejar las imagenes de ese canal, el nombre 
    de la figura respectiva, y la señal de la ventana en dicho instante de tiempo. 
    Calcula la transformada continua wavelet Morlet compleja, y entrega la imagen a partir de ella.
    Entrega un void, pues solo genera imagenes. 
    """
    fig = plt.figure(frameon=False)
    fig = plt.gcf()
    DPI = fig.get_dpi()
    #249
    fig.set_size_inches(235.0/float(DPI),235.0/float(DPI))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    scales = np.arange(5,76)
    wavelet = 'cmor2.0-0.5'
    #cmorB-C, where B is the bandwidth and C is the center frequency 
    cwtmatr, freqs = pywt.cwt(signal, scales, wavelet )
    # Wavelet power spectrum
    #By default the origin is upper. we want it to start from the bottom. aspecto 'auto'
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    #vmi.\n=0, vmax=255 cm.gray jet
    ax.imshow((np.abs(cwtmatr)), cmap=cm.gray, origin='lower',aspect='auto')
    #img_gray = myImage2gray(ax)
    #gray = (myimg)
    #ax.imshow(gray, cmap=cm.gray, origin='lower',aspect='auto')
    extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    #fig.savefig(rutafigChannel+figname, dpi = 'figure', bbox_inches=extent,transparent=True, pad_inches=0)
    #plt.close(fig)
    return fig

### fig2data: recibe una figura, lo transforma en un arreglo numoy rgba y lo retorna.

In [7]:
def fig2data( fig ):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw()
 
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.frombuffer(fig.canvas.tostring_argb(), dtype=np.uint8 )
    buf.shape = ( w, h,4 )
    
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll( buf, 3, axis = 2 )
    #Hay que cerrar la figura!!!! Importante. No borrar
    plt.close(fig)
    return buf

### fig2img: convierte una figura en una imagen a traves de pil

In [8]:
def fig2img( fig ):
    """
    @brief Convert a Matplotlib figure to a PIL Image in RGBA format and return it
    @param fig a matplotlib figure
    @return a Python Imaging Library ( PIL ) image
    """
    # put the figure pixmap into a numpy array
    buf = fig2data(fig)
    w, h, d = buf.shape
    img = Image.frombytes( "RGBA", ( w ,h ), buf.tostring( ) )
    new_width  = 256
    new_height = 256
    img2 = img.resize((new_width, new_height), Image.ANTIALIAS)
    #imgfinal = img.convert('L')
    return img2

### myImage2gray convierte una imagen rgba en una imagen en escala de grises ( con una sola dimension!!)

In [9]:
from skimage.color import rgba2rgb,rgb2gray

def myImage2gray(fig):
    #, rutafigChannel, figname):
    img_rgb = rgba2rgb(fig)
    img_gray = rgb2gray(img_rgb)
    return img_gray

# La manera de llamar entonces es la siguiente: 
myImage2gray(fig2img(wave2Image(theWin))) : theWin es la ventana, wave2Image la transforma en figura wavelet, fig2img la transforma en numpy array e imagen rgba, y finalmente myImage2Gray la transforma a escala de grises.

### Este metodo genera imagenes de las ventanas traslapadas para un canal

In [11]:
#rutafig = '/home/cmanalisis/1_SergioEhijo/Base de Datos/C14_Alex/'
#sig=lfp.LFP[2][index:]
#cont=0
#img = myImage2gray(fig2img(wave2Image(lfp.LFP[0][index:index+1000])))
#myImage2gray(wave2Image(lfp.LFP[0][index:index+1000]), rutafig, 'hola2')
#plt.imshow(fig2img(fig1))
#result = Image.fromarray((img * 255).astype(np.uint8))
#result.save(rutafig+'Ventana' + str(3) + 'CH1'+'.png')

lenLFP = np.arange(44)
winSize = 1000
step = 700
Hechos = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
         ,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
         ,33,34,35,36,37,38,39,40,41]
for i in range(44,45):
    it = lenLFP[i]
    Canal = lfp.Channel[it]
    Area = lfp.Area[it]
    sig = lfp.LFP[it][index:]
    rutafig = '/home/cmanalisis/1_SergioEhijo/Base de Datos/C14_Alex/'+ Canal +'/'
    ventanas = slidingWindow(sig,winSize,step)
    for j in tqdm(range(len(ventanas))):
        figname = 'Ventana_'  + str(j)+ '_' + Canal + '_' + Area +'.png'
        theWin = ventanas[j]
        img= myImage2gray(fig2img(wave2Image(theWin)))
        result = Image.fromarray((img * 255).astype(np.uint8))
        result.save(rutafig+figname)
    print('Number {} of Area {} and Channel {} is done'.format(it, Area, Canal))

IndexError: index 44 is out of bounds for axis 0 with size 44

In [ ]:
#lenLFP = np.arange(44)
#winSize = 1000
#step = 700
#Hechos = []
#for i in range(0,1):
#    it = lenLFP[i]
#    Canal = lfp.Channel[it]
#    sig = lfp.LFP[it][index:]
#    rutafig = '/home/cmanalisis/1_SergioEhijo/Base de Datos/C14_Alex/'+ Canal +'/'
#    ventanas = slidingWindow(sig,winSize,step)
#    for j in tqdm(range(len(ventanas))):
#        figname = 'Ventana_'+ str(j)+'_'+Canal+'.png'
#        theWin = ventanas[j]
#        wave2Image(rutafig, figname, theWin)

In [ ]:
import imageio
%matplotlib inline
#filena = '/home/cmanalisis/1_SergioEhijo/Base de Datos/Test.png'
im = imageio.imread('/home/cmanalisis/1_SergioEhijo/Base de Datos/C14_Alex/2out.png')
imgplot = plt.imshow(im, cmap= cm.gray)
#plt.gca().set_xticks([])
#plt.gca().set_yticks([])
#plt.gca().spines['top'].set_visible(False)
#plt.gca().spines['left'].set_visible(False)
#plt.gca().spines['right'].set_visible(False)
#plt.gca().spines['bottom'].set_visible(False)
#plt.show()
im.shape

### A continuacion, se obtiene la informacion necesaria del arduino a partir del canal adc1

In [ ]:
# definir el root path del directorio donde estan los archivos
# El backslash al final de 
### Control Salino
#rootpath = '/home/cmanalisis/1_SergioEhijo/Ratas/ControlSalino/CA_EC03_NS__2018-09-05_13-14-43/' 
# Alex
rootpath = '/home/cmanalisis/1_SergioEhijo/Ratas/C14_Alex/CA_EC03_C14_PreQx__2018-09-12_15-12-28/'
fullpath = rootpath + '*ADC*.continuous'
lista = glob(fullpath)
lista = sorted(lista)
data= OE.loadContinuous(lista[0])
data

In [ ]:
arduino = data['data']
orSampleRate = int(data['header']['sampleRate'])
desiredSampleRate = 1000.0
# hace el donwsampling desde el sampling original hasta 1000 Hz
downsamplefactor = int(orSampleRate/desiredSampleRate)
# multiplo de downsampling factor
multiplo_dsf = int(len(arduino)/downsamplefactor)
# crear array de n x downsamplefactor para promediar 
arduino_final = arduino[0:downsamplefactor*multiplo_dsf].reshape(-1, downsamplefactor)
# promedio y con esto el downsampling esta finalizado
arduino_final = arduino_final.mean(axis=1)
# crear eje de tiempo para la señal down-sampleada
t2 = np.arange(1/desiredSampleRate,len(arduino_final)/desiredSampleRate + 1/desiredSampleRate,1/desiredSampleRate)

In [ ]:
fig, ax = plt.subplots()
#sigg = lfp.LFP[0]
#plt.plot(t2[ind:],arduino[ind:])
plt.plot(t2,arduino_final)
#plt.plot(timeLFP[ind:],sigg[ind:])
#plt.xlabel('Time [ms]')
#plt.ylabel('Voltage [V]')
#plt.title('Arduino signal in time')
#
#ind2 = ind/30
#print(len(arduino[ind:]))
#print(len(sigg[ind:])/1000)

In [ ]:
#wave2Image(rutafig, 'test11', ventanas[11])
len(ventanas)

In [ ]:
fig,(ax1,ax2) = plt.subplots(2, 1, sharex='row')
sig=lfp.LFP[0][0:1000]
ax1.plot(sig)
ax1.axes.get_xaxis().set_visible(False)
scales = np.arange(5,75+1)
#cmorB-C, where B is the bandwidth and C is the center frequency 
cwtmatr, freqs = pywt.cwt(sig, scales, 'cmor2.0-0.5')
# Wavelet power spectrum
#By default the origin is upper. we want it to start from the bottom.
Colorsinos = ax2.imshow((np.abs(cwtmatr)), cmap = 'jet', origin='lower',aspect='auto', extent = [0,1,
                                                                        0,70])
cbar1 = plt.colorbar(Colorsinos, orientation = 'horizontal')
cbar1.ax.set_xlabel('Power [mV**2]')
plt.rcParams["figure.figsize"] =(8,6)
plt.xlabel('Time [Seconds]')
plt.ylabel('Frequency [Hz]')
plt.show()

### Por inspección visual se busca el momento en que el arduino deja de hacer el setup y empieza con el LED, lo cual se puede ver en el primer pulso solo, mas ancho que los demás, y que presenta un leve bajón. Se compruba comparando el ancho de ese pulso, desde el bajón hasta el final, con los demás pulsos que siguen; todos poseen el mismo ancho y encienden el LED. 
Ese bajón corresponde al cero de los tiempos para LFPs y videos y por ende es de interés !

###### Metodo para obtener las frecuencias a partir de las escalas de la t c wavelet

In [ ]:
dt = 0.001 #1000 hz sampling
frequencies = pywt.scale2frequency('cmor2.0-0.5', [5,6,7,8,10,12,14,16,20,25,33,40,75]) / dt
print(frequencies)
print(scales)


#### Para ver la forma de transformada en el tiempo

In [ ]:
wave = pywt.ContinuousWavelet('cmor2.0-0.5')
psi, x = wave.wavefun()
plt.plot(x,psi)